In [2]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from laspy.file import File
import pptk
from sklearn import preprocessing

%matplotlib inline

In [3]:
from pathlib import Path

In [4]:
!ls -al "/Users/ethanchen/The University of Melbourne/Trent Penman - Sites"

total 11783744
drwxr-xr-x  31 ethanchen  staff         992 16 Apr 20:15 .
drwxr-xr-x@  5 ethanchen  staff         160 16 Apr 20:15 ..
-rw-------   1 ethanchen  staff          63 16 Apr 20:15 .849C9593-D756-4E56-8D6E-42412F2A707B
-rw-r--r--@  1 ethanchen  staff        8196 17 Apr 03:08 .DS_Store
-rwxr-xr-x   1 ethanchen  staff   107870208 25 Oct 10:40 101 Sharps.e57
-rwxr-xr-x   1 ethanchen  staff   113014784 25 Oct 15:31 104 Sharps.e57
-rwxr-xr-x   1 ethanchen  staff   159873024 25 Oct 15:33 118 Dunse.e57
-rwxr-xr-x   1 ethanchen  staff   139356160 25 Oct 15:33 119 Dunse.e57
-rwxr-xr-x   1 ethanchen  staff  2750810112 18 Oct 13:50 126 Wye.e57
-rwxr-xr-x   1 ethanchen  staff   136613888 25 Oct 11:12 184 Big Hill.e57
-rwxr-xr-x   1 ethanchen  staff   117312512 25 Oct 15:27 57 Bambra aireys.e57
-rwxr-xr-x   1 ethanchen  staff   130904064 22 Oct 13:38 60 Bambra aireys.e57
-rwxr-xr-x   1 ethanchen  staff   153603072 22 Oct 09:52 84ish Seaview.e57
-rwxr-xr-x   1 ethanchen  staff   199784448 

In [5]:
file_root_path = Path("/Users/ethanchen/The University of Melbourne/Trent Penman - Sites")

file_root_path

PosixPath('/Users/ethanchen/The University of Melbourne/Trent Penman - Sites')

In [6]:
las_format_folder = file_root_path / 'las_format'
print(las_format_folder)

/Users/ethanchen/The University of Melbourne/Trent Penman - Sites/las_format


In [7]:
!ls -al "/Users/ethanchen/The University of Melbourne/Trent Penman - Sites/las_format"

total 26722560
drwxr-xr-x  30 ethanchen  staff         960  9 Apr 11:00 .
drwxr-xr-x  31 ethanchen  staff         992 16 Apr 20:15 ..
-rw-r--r--@  1 ethanchen  staff        6148  9 Apr 09:45 .DS_Store
-rwxr-xr-x   1 ethanchen  staff   243454541 29 Oct 13:07 101 sharps.las
-rwxr-xr-x   1 ethanchen  staff   255065099 29 Oct 13:08 104 sharps.las
-rwxr-xr-x   1 ethanchen  staff   360823765 29 Oct 13:08 118 Dunse.las
-rwxr-xr-x   1 ethanchen  staff   314518349 29 Oct 13:09 119 Dunse.las
-rwxr-xr-x   1 ethanchen  staff  6208539661 27 Oct 15:44 126_Wye.las
-rwxr-xr-x   1 ethanchen  staff   308328241 29 Oct 13:13 184 Big Hill.las
-rwxr-xr-x   1 ethanchen  staff   295441629 29 Oct 13:02 60 Bambra aireys.las
-rwxr-xr-x   1 ethanchen  staff   346674155 29 Oct 13:04 84ish seaview.las
-rwxr-xr-x   1 ethanchen  staff   450903161 27 Oct 15:01 85 Seaview.las
-rwxr-xr-x   1 ethanchen  staff   362439275 29 Oct 13:06 94 seaview.las
-rwxr-xr-x   1 ethanchen  staff   324128891 29 Oct 13:06 95 seaview.las
-

#### What are the potential features available features

In [40]:
#What are the potential features available features
file_to_explore = 'SP01.las'
inFile = File(las_format_folder/file_to_explore, mode='r')
dir(inFile)

['Blue',
 'Classification',
 'Green',
 'Intensity',
 'Key_Point',
 'Raw_Classification',
 'Red',
 'Synthetic',
 'Withheld',
 'X',
 'Y',
 'Z',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_evlrs',
 '_header',
 '_mode',
 '_reader',
 '_vlrs',
 'addProperty',
 'assertWriteMode',
 'blue',
 'byte_offset_to_waveform_data',
 'classification',
 'classification_flags',
 'close',
 'define_new_dimension',
 'doc',
 'edge_flight_line',
 'extra_bytes',
 'filename',
 'flag_byte',
 'get_blue',
 'get_byte_offset_to_waveform_data',
 'get_classification',
 'get_classification_flags',
 'get_edge_flight_line',
 'get_ext

#### Look at a single file

In [41]:
print(np.unique(inFile.get_classification()))
print(np.unique(inFile.get_classification_flags()))
print(np.unique(inFile.get_key_point()))
print(np.unique(inFile.Raw_Classification))
print(np.unique(inFile.Classification))
print(np.unique(inFile.Intensity))
print(len(np.unique(inFile.Green)))
print(len(np.unique(inFile.Blue)))
print(len(np.unique(inFile.Red)))

[0]
[0]
[0]
[0]
[0]
[0]
256
256
256


##### Looping throught all the files

In [36]:
from os import listdir
from os.path import isfile, join
mypath = "/Users/ethanchen/The University of Melbourne/Trent Penman - Sites/las_format"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f)) and f.endswith('las')]
onlyfiles

['SP01.las',
 '94 seaview.las',
 'SP03_Stony.las',
 'carpark.las',
 '126_Wye.las',
 '60 Bambra aireys.las',
 '184 Big Hill.las',
 'SP13_Garvey.las',
 '119 Dunse.las',
 '95 seaview.las',
 'SP04_Gentle_Annie_Creek.las',
 '85 Seaview.las',
 'SP09_Coalmine_ck.las',
 'SP06_Gentle_Annie.las',
 'SP08 Bambra Aireys nearIronbark.las',
 '84ish seaview.las',
 'SP10_Moggs_Creek.las',
 'SP12_Stony.las',
 '118 Dunse.las',
 'SP11_Seaview_extra.las',
 '104 sharps.las',
 '101 sharps.las',
 'SP02 Big Hill.las',
 'SP08_Bambra_Aireys_nearIronbark.las',
 'SP14_Garvey.las']

In [37]:


for file in onlyfiles:
    file_to_explore = file
    print(file)
    inFile = File(las_format_folder/file_to_explore, mode='r')
    headerformat = inFile.header.header_format
    print(np.unique(inFile.get_classification()))
    print(np.unique(inFile.get_classification_flags()))
    print(np.unique(inFile.get_key_point()))
    print(np.unique(inFile.Raw_Classification))
    print(np.unique(inFile.Classification))
    print(np.unique(inFile.Intensity))
    print(len(np.unique(inFile.Green)))
    print(len(np.unique(inFile.Blue)))
    print(len(np.unique(inFile.Red)))
    print('-'*50)

SP01.las
[0]
[0]
[0]
[0]
[0]
[0]
256
256
256
--------------------------------------------------
94 seaview.las
[0]
[0]
[0]
[0]
[0]
[0]
256
256
256
--------------------------------------------------
SP03_Stony.las
[0]
[0]
[0]
[0]
[0]
[0]
256
256
256
--------------------------------------------------
carpark.las
[0]
[0]
[0]
[0]
[0]
[0]
256
256
256
--------------------------------------------------
126_Wye.las
[0]
[0]
[0]
[0]
[0]
[0]
256
256
256
--------------------------------------------------
60 Bambra aireys.las
[0]
[0]
[0]
[0]
[0]
[0]
256
256
256
--------------------------------------------------
184 Big Hill.las
[0]
[0]
[0]
[0]
[0]
[0]
256
256
256
--------------------------------------------------
SP13_Garvey.las
[0]
[0]
[0]
[0]
[0]
[0]
256
256
256
--------------------------------------------------
119 Dunse.las
[0]
[0]
[0]
[0]
[0]
[0]
256
256
256
--------------------------------------------------
95 seaview.las
[0]
[0]
[0]
[0]
[0]
[0]
256
256
256
-----------------------------------

In [38]:
# [x for x in las_format_folder.iterdir() if x.is_file()][:4]